In [13]:
import numpy as np
import pandas as pd
import warnings
from scipy.optimize import minimize_scalar
from anndata import AnnData
import os
from pathlib import Path
import gzip
import pandas as pd
from cmapPy.pandasGEXpress import parse

pr_gene_id
780       1
26046     1
57711     1
641339    1
256051    1
         ..
22797     1
7369      1
4004      1
23546     1
60        1
Name: count, Length: 12328, dtype: int64

In [61]:


# 1. 路径配置
PROJECT_ROOT = Path("e:/科研/Models/drugreflector")  # 根据实际路径调整
DATA_DIR = PROJECT_ROOT / "datasets"
GCTX_FILE = DATA_DIR / "GSE70138_Broad_LINCS_Level5_COMPZ_n118050x12328_2017-03-06.gctx"

# Decompress the .gz file if the decompressed file does not already exist
if not GCTX_FILE.exists():
    import gzip
    import shutil
    with gzip.open(f"{GCTX_FILE}.gz", "rb") as f_in:
        with open(GCTX_FILE, "wb") as f_out:
            shutil.copyfileobj(f_in, f_out)
CELL_INFO = DATA_DIR / "GSE70138_Broad_LINCS_cell_info_2017-04-28.txt.gz"
GENE_INFO = DATA_DIR / "GSE70138_Broad_LINCS_gene_info_2017-03-06.txt.gz"

# 2. 元数据读取
def read_gzip_tsv(path):
    with gzip.open(path, "rt") as fh:
        return pd.read_csv(fh, sep="\t")

gene_info = read_gzip_tsv(GENE_INFO)
cell_info = read_gzip_tsv(CELL_INFO)
print(gene_info.head())
print(cell_info.head())

# 3. 读取 GCTX（可能耗内存，可先抽样）
gctx_parser = parse.parse(str(GCTX_FILE), rid=None, cid=None)  # rid/cid 可传列表限制基因/签名
expr_df = gctx_parser.data_df  # 行: genes, 列: signatures
print(expr_df.shape)

# 4. landmark 筛选
landmark_genes = gene_info.loc[gene_info["pr_is_lm"] == 1, "pr_gene_id"]
landmark_genes = landmark_genes.astype(str)
# landmark_genes is a pd.Series
expr_df = expr_df.loc[landmark_genes.values]
print("After landmark filter:", expr_df.shape)

breakpoint()
# 5. 预处理示例：clip_rescale_rows
def _norm_func(data, clip, target_std):
    """Helper function called in `clip_rescale_rows`."""
    return lambda norm: (np.std(np.clip(data / norm, -clip, clip)) - target_std) ** 2


def clip_rescale_rows(X, clip, target_std, bounds=(0, 1e3)):
    """For each row of X, rescale and then clip the values such that the
    resulting standard deviation is as close to possible as to target_std. This
    function mutates X itself.

    Params
    ------
    X : 2d ndarray
    clip : float
    target_std : float
    bounds : (float, float)
        bounds of optimization for finding scaling constant
    """
    n = X.shape[0]
    norm = np.zeros(n)
    for i in range(n):
        ms = minimize_scalar(_norm_func(X[i], clip, target_std), method='Bounded', bounds=bounds)
        assert ms.success, f'Unable to rescale row {i}.'
        norm[i] = ms.x

    np.divide(X, norm[:, np.newaxis], out=X)
    np.clip(X, -clip, clip, out=X)
expr_matrix = expr_df.to_numpy(dtype="float32")
clip_rescale_rows(expr_matrix, clip=2, target_std=1)
expr_df.loc[:, :] = expr_matrix


gctx_parser.col_metadata_df.head()

# 6. 保存/后续处理
expr_df = expr_df.T  # 根据需求转置为签名×基因
print(expr_df)

   pr_gene_id pr_gene_symbol                                pr_gene_title  \
0         780           DDR1  discoidin domain receptor tyrosine kinase 1   
1        7849           PAX8                                 paired box 8   
2        2978         GUCA1A               guanylate cyclase activator 1A   
3        2049          EPHB3                              EPH receptor B3   
4        2101          ESRRA              estrogen related receptor alpha   

   pr_is_lm  pr_is_bing  
0         1           1  
1         1           1  
2         0           0  
3         0           1  
4         0           1  
    cell_id  cell_type base_cell_id precursor_cell_id  \
0      A375  cell line         A375              -666   
1  A375.311  cell line         A375              A375   
2      A549  cell line         A549              -666   
3  A549.311  cell line         A549              A549   
4      A673  cell line         A673              -666   

                                      

d:\ProgramData\anaconda3\envs\chemprop\Lib\site-packages\cmapPy\pandasGEXpress\parse_gctx.py:275: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  meta_df = meta_df.apply(lambda x: pd.to_numeric(x, errors="ignore"))
d:\ProgramData\anaconda3\envs\chemprop\Lib\site-packages\cmapPy\pandasGEXpress\parse_gctx.py:275: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  meta_df = meta_df.apply(lambda x: pd.to_numeric(x, errors="ignore"))


(12328, 118050)
After landmark filter: (978, 118050)
rid                         780      7849      6193        23      9552  \
cid                                                                       
REP.A001_A375_24H:A03  2.000000  0.075918 -2.000000 -0.286138 -0.468712   
REP.A001_A375_24H:A04 -0.476658  0.403548 -1.259336 -0.865485 -0.806891   
REP.A001_A375_24H:A05 -0.712984 -1.001199 -0.897927  0.553344  0.339323   
REP.A001_A375_24H:A06  0.728779 -0.782359 -0.033380 -0.084206  0.674925   
REP.A001_A375_24H:A07  0.821030 -0.300829 -1.446070  0.441786  0.473586   
...                         ...       ...       ...       ...       ...   
LJP007_SKL_24H:O13     2.000000  2.000000  2.000000  2.000000 -2.000000   
LJP007_SKL_24H:O14     2.000000  2.000000  2.000000  2.000000 -1.313821   
LJP007_SKL_24H:O24     2.000000  0.650449  2.000000  2.000000 -1.935318   
LJP007_SKL_24H:P24     2.000000  2.000000 -1.586303 -2.000000  1.212872   
LJP007_SKL_24H:C19     2.000000  2.000000  0.48

In [57]:
gctx_parser.row_metadata_df.head()

rhd
rid
780
7849
2978
2049
2101


In [60]:
print(expr_df)

cid    REP.A001_A375_24H:A03  REP.A001_A375_24H:A04  REP.A001_A375_24H:A05  \
rid                                                                          
780                 2.000000              -0.476658              -0.712984   
7849                0.075918               0.403548              -1.001199   
6193               -2.000000              -1.259336              -0.897927   
23                 -0.286138              -0.865485               0.553344   
9552               -0.468712              -0.806891               0.339323   
...                      ...                    ...                    ...   
5467               -2.000000               0.129037              -0.158048   
2767                0.886145               0.422537               0.394286   
23038              -0.452821               1.131562              -0.269333   
57048               1.931802              -0.568680              -0.482195   
79716              -0.557628              -0.729979             

In [67]:
from datasets import load_dataset
# Load dataset wholescale and download 
ds = load_dataset("tahoebio/Tahoe-100m", split="train")
print(ds)

Resolving data files:   0%|          | 0/3388 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/3388 [00:00<?, ?it/s]

train-00000-of-03388.parquet:   0%|          | 0.00/71.2M [00:00<?, ?B/s]

train-00001-of-03388.parquet:   0%|          | 0.00/75.0M [00:00<?, ?B/s]

train-00002-of-03388.parquet:   0%|          | 0.00/75.2M [00:00<?, ?B/s]

KeyboardInterrupt: 